In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer
from transformers import AlbertModel

In [2]:
from datasets import load_dataset

dataset = load_dataset("tweet_eval", "stance_feminist")

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/597 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/285 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/67 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to C:/Users/User/.cache/huggingface/datasets/tweet_eval/stance_feminist/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
dataset["train"]

Dataset({
    features: ['text', 'label'],
    num_rows: 597
})

In [4]:
dataset["validation"]

Dataset({
    features: ['text', 'label'],
    num_rows: 67
})

In [5]:
dataset["validation"]

Dataset({
    features: ['text', 'label'],
    num_rows: 67
})

In [6]:
# Load a pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")


In [7]:
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

In [8]:
# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Get the input_ids from the tokenized dataset for train, validation, and test sets
train_input_ids = [example['input_ids'] for example in tokenized_dataset['train']]
val_input_ids = [example['input_ids'] for example in tokenized_dataset['validation']]
test_input_ids = [example['input_ids'] for example in tokenized_dataset['test']]

# Convert input_ids to PyTorch tensors
train_input_ids = torch.tensor(train_input_ids, dtype=torch.long)
val_input_ids = torch.tensor(val_input_ids, dtype=torch.long)
test_input_ids = torch.tensor(test_input_ids, dtype=torch.long)

# Concatenate the input_ids tensors along the first dimension
input_ids = torch.cat((train_input_ids, val_input_ids, test_input_ids), dim=0)


Map:   0%|          | 0/597 [00:00<?, ? examples/s]

Map:   0%|          | 0/285 [00:00<?, ? examples/s]

Map:   0%|          | 0/67 [00:00<?, ? examples/s]

In [9]:
# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Get the attention_masks from the tokenized dataset for train, validation, and test sets
train_attention_mask = [example['attention_mask'] for example in tokenized_dataset['train']]
val_attention_mask = [example['attention_mask'] for example in tokenized_dataset['validation']]
test_attention_mask = [example['attention_mask'] for example in tokenized_dataset['test']]

# Convert attention_mask to PyTorch tensors
train_attention_mask = torch.tensor(train_attention_mask, dtype=torch.long)
val_attention_mask = torch.tensor(val_attention_mask, dtype=torch.long)
test_attention_mask = torch.tensor(test_attention_mask, dtype=torch.long)

# Concatenate the input_ids tensors along the first dimension
attention_masks = torch.cat((train_attention_mask, val_attention_mask, test_attention_mask), dim=0)


Map:   0%|          | 0/597 [00:00<?, ? examples/s]

Loading cached processed dataset at C:\Users\User\.cache\huggingface\datasets\tweet_eval\stance_feminist\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343\cache-d6fc5813b617ce88.arrow
Loading cached processed dataset at C:\Users\User\.cache\huggingface\datasets\tweet_eval\stance_feminist\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343\cache-4a19f2a3383c5d78.arrow


In [10]:
print(input_ids)

tensor([[    2,    13,     1,  ...,     0,     0,     0],
        [    2,    13,     1,  ...,     0,     0,     0],
        [    2,    13,  5256,  ...,     0,     0,     0],
        ...,
        [    2,    31,   765,  ...,     0,     0,     0],
        [    2,    13,     1,  ...,     0,     0,     0],
        [    2,  6926, 10817,  ...,     0,     0,     0]])


In [11]:
print(attention_masks)

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


In [12]:
# Split the dataset into train, validation, and test sets
train_dataset = tokenized_dataset["train"]
val_dataset = tokenized_dataset["validation"]
test_dataset = tokenized_dataset["test"]

In [13]:
# Convert the datasets to PyTorch tensors
train_dataset = train_dataset.remove_columns(["text"])
val_dataset = val_dataset.remove_columns(["text"])
test_dataset = test_dataset.remove_columns(["text"])

In [14]:
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [15]:
class StanceFeministModel(nn.Module):
    def __init__(self, num_classes=3):
        super(StanceFeministModel, self).__init__()
        self.albert = AlbertModel.from_pretrained('albert-base-v2')
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.albert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.albert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        return logits

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [17]:
# instantiate your model
stance_feminist_model = StanceFeministModel(num_classes=3).to(device)

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.LayerNorm.weight', 'predictions.bias', 'predictions.LayerNorm.bias', 'predictions.dense.bias', 'predictions.decoder.bias', 'predictions.dense.weight', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
# define your loss function
criterion = nn.CrossEntropyLoss()

# define your optimizer
optimizer = torch.optim.Adam(stance_feminist_model.parameters(), lr=0.01)

In [19]:
# set up your training, validation, and test dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

In [20]:
for batch in train_loader:
    print(batch)


{'label': tensor([1, 0, 1, 0, 0, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1]), 'input_ids': tensor([[   2, 9321, 7617,  ...,    0,    0,    0],
        [   2,  148,   72,  ...,    0,    0,    0],
        [   2, 9704,   18,  ...,    0,    0,    0],
        ...,
        [   2,   48, 6926,  ...,    0,    0,    0],
        [   2,  221,   22,  ...,    0,    0,    0],
        [   2,   13,    1,  ...,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
{'label': tensor([1, 2, 0, 1, 1, 0, 0, 1, 2, 1, 2, 1, 2, 1, 1, 0]), 'input_ids': tensor([[    2, 22890,    25,  ...,     0,     0,     0],
        [    2,    31,  1376,  ...,     0,     0,     0],
        [    2,    13,     1,  ...,     0,     0,     0],
        ...,
        [    2,    76,   144,  ...,     0,     0,     0],
        [    2,    31,  2321,

In [21]:
from tqdm import tqdm
num_epochs = 1
for epoch in range(num_epochs):
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        # get the inputs
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = stance_feminist_model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

        # print running loss for each batch
        running_loss += loss.item()
        avg_loss = running_loss / len(train_loader)
        avg_acc = correct_predictions / total_predictions
        tqdm.write(f'Train Loss: {avg_loss:.3f}, Train Acc: {avg_acc:.3f}', end='\r')
    tqdm.write(f'Epoch {epoch+1}, Train Loss: {avg_loss:.3f}, Train Acc: {avg_acc:.3f}')

    print(f"Epoch {epoch+1} finished")


Epoch 1/1:   3%|▎         | 1/38 [00:43<26:58, 43.74s/it]

Epoch 1/1:   5%|▌         | 2/38 [01:08<19:38, 32.72s/it]

Epoch 1/1:   8%|▊         | 3/38 [02:00<24:04, 41.26s/it]

Epoch 1/1:  11%|█         | 4/38 [02:57<26:54, 47.48s/it]

Epoch 1/1:  13%|█▎        | 5/38 [03:41<25:32, 46.44s/it]

...

Epoch 1/1:  92%|█████████▏| 35/38 [16:28<01:17, 25.89s/it]

Epoch 1/1:  95%|█████████▍| 36/38 [17:10<01:01, 30.75s/it]

Epoch 1/1:  97%|█████████▋| 37/38 [17:49<00:33, 33.17s/it]

Epoch 1/1: 100%|██████████| 38/38 [18:04<00:00, 28.54s/it]

Epoch 1, Train Loss: 2.512, Train Acc: 0.414
Epoch 1 finished


In [22]:
# Validation loop
with torch.no_grad():
    stance_feminist_model.eval()  # Set the model to evaluation mode
    valid_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    for batch in val_loader:
        # get the inputs
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # forward
        outputs = stance_feminist_model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)

        # calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

        # calculate running loss
        valid_loss += loss.item()

    avg_loss = valid_loss / len(val_loader)
    avg_acc = correct_predictions / total_predictions
    print(f'Validation Loss: {avg_loss:.3f}, Validation Acc: {avg_acc:.3f}')

Validation Loss: 2.522, Validation Acc: 0.194


In [23]:

# Test loop
with torch.no_grad():
    stance_feminist_model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    for batch in test_loader:
        # get the inputs
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # forward
        outputs = stance_feminist_model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)

        # calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

        # calculate running loss
        test_loss += loss.item()

    avg_loss = test_loss / len(test_loader)
    avg_acc = correct_predictions / total_predictions
    print(f'Test Loss: {avg_loss:.3f}, Test Acc: {avg_acc:.3f}')


Test Loss: 2.192, Test Acc: 0.154


In [24]:
# save the model
torch.save(stance_feminist_model.state_dict(), 'Models/stance_feminist_model.pth')
